# Algo 2 – Python

## Cours 8 – Web côté serveur

###  Master Humanités Numériques du CESR

La semaine passée nous avons vu comment émettre des requêtes à une API à l’aide du module `requests`

Nous allons voir maintenant comment envoyer des réponses à l’aide d’un serveur web

![serveur web](https://developer.mozilla.org/fr/docs/Learn/Common_questions/Web_mechanics/What_is_a_web_server/web-server.svg)  
<small>[https://developer.mozilla.org/fr/docs/Learn/Common_questions/Web_mechanics/What_is_a_web_server](https://developer.mozilla.org/fr/docs/Learn/Common_questions/Web_mechanics/What_is_a_web_server)</small>

Un serveur HTTP reçoit des requêtes et renvoie des réponses.

Dans le cas d’un serveur web *statique*, les réponses sont des fichiers html et des fichiers css, javascript, image, … les requêtes HTTP contiennent des noms de chemins qui peuvent correspondre à l’arborescence des dossiers *servis* par le serveur 

✍️ Exercice ✍️

- Dans un dossier `web`, créez deux fichiers html (`index.html` et `about.html`). Chaque fichier devra contenir un lien vers l’autre, pour le reste peu importe le contenu.

- Ouvrez un terminal, déplacez vous dans le dossier `web` (`cd web`)
- Tapez la commande `python3 -m http.server` 
- Connectez-vous à `http://localhost:8000`
- (ctrl-C pour arrềter)

Bravo ! Vous avez lancé un serveur web local. Il va répondre aux requêtes de type `GET` à condition que le chemin demandé corresponde à un fichier de votre dossier `web`.

Essayez de demander par exemple `http://localhost:8000/spaghetti` et voyez ce qu’il se passe.

[`http.server`](https://docs.python.org/3/library/http.server.html) est un serveur web minimaliste en Python. Il peut être utilisé en développement, à des fins de test ou de prototypage mais pas en production.


### Flask

[Flask](https://flask.palletsprojects.com/) est un framework Python minimaliste qui permet de construire des applications web rapidement.

Dans la même veine il y aussi [CherryPy](https://cherrypy.dev/), [Pylons](https://pylonsproject.org/) ou encore [FastAPI](https://fastapi.tiangolo.com/) qui est plutôt dédié à la construction d’API.

[Django](https://www.djangoproject.com/) dont vous avez entendu parler a une approche différente. Il est plus complet, intègre beaucoup de fonctionnalités et plus adapté aux projets de grande taille.

Nous allons reprendre beaucoup d’éléments de [https://flask.palletsprojects.com/en/3.0.x/quickstart/](https://flask.palletsprojects.com/en/3.0.x/quickstart/). Le mieux est de s’y référer directement.

#### Installation

Pour installer Flask et ses dépendances : 
`python3 -m pip install Flask --user` ou `pip install Flask --user`

#### Première page



In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello():
    return "<p>Bonjour !</p>"

- Sauvegardez ces lignes dans un fichier `hello.py`
- Dans un terminal tapez : `flask --app hello run`
- Connectez-vous à http://localhost:5000

La logique est différente de celle d’`http.server`. Ici vous ne servez pas des fichiers, vous définissez des *routes* (avec le décorateur `@app.route`) et vous générez des réponses. En cela `Flask` est un serveur dynamique, la réponse apportée pourra dépendre de la requête et de ses éventuels paramètres.

### Variables

Vous pouvez utiliser des routes avec des variables :

In [1]:
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello():
    return "<p>Bonjour !</p>"

@app.route("/hello/<prenom>")
def hello_2(prenom):
    return f"<p>Bonjour {prenom} !</p>"

Pour tester vos changements vous devez arrêter (ctrl-C) et relancer le serveur. Vous pouvez vous épargner cette manipulation en utilisant la commande :  
`flask --app hello --debug run`

Connectez vous à `http://localhost:5000` puis `http://localhost:5000/hello/toi` puis amusez-vous à changer la valeur de la variable pour voir le résultat.

### Templates

Pour le moment nos réponses ne sont pas très correctes. Un élément `<p>` ne fait pas une page html valide.  
`Flask`, comme beaucoup de framework web, permet d’utiliser des *templates* (des patrons) de page html dans lequels on insère des variables et des sections dynamiques.  
`Flask` utilise le moteur de template [Jinja](https://jinja.palletsprojects.com/en/3.1.x/templates/)

Notre fichier `hello.py` devient :

In [ ]:
from flask import Flask
from flask import render_template

app = Flask(__name__)

@app.route('/')
def hello():
    return "<p>Bonjour !</p>"


@app.route('/hello/')
@app.route('/hello/<prenom>')
def hello_2(prenom=None):
    return render_template('hello.html', name=prenom)

Vous ajoutez à votre dossier `web` le fichier `templates/hello.html` qui contiendra :

In [ ]:
<!doctype html>
<html>

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Hello</title>
</head>

<body>
    {% if name %}
    <h1>Bonjour {{ name }} !</h1>
    {% else %}
    <h1>Bonjour tout le monde !</h1>
    {% endif %}
</body>

</html>

### Requêtes

Évidemment `Flask` permet de manipuler des requêtes HTTP avec des paramètres.  
On va ajouter une route qui répond à une requête utilisant les méthodes `GET` ou `POST` (utilisez l’inspecteur de votre navigateur pour voir les en-têtes de requête et de réponses HTTP)

In [ ]:
@app.route('/form/', methods=['GET', 'POST'])
def hello_form():
    if request.method == 'POST':
        # Get the value of the 'firstname' field from the form
        firstname = request.form['firstname']
        return render_template('form_result.html', firstname=firstname)
    return render_template('form.html')

Les templates ce sera :
- `form.html`
Attention il faut que le chemin indiqué en valeur de l’attribut `action` corresponde à votre route

In [ ]:
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Hello</title>
</head>
<body>
    <h2>Quel est votre prénom ?</h2>
    <form method="POST" action="/form/">
        <input type="text" name="firstname" required>
        <button type="submit">Submit</button>
    </form>
</body>
</html>


- `form_result.html`

In [ ]:
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Hello</title>
</head>
<body>
    <h2>Bonjour {{ firstname }}</h2>
</body>
</html>

### Retour à nos prénoms

Notre objectif est de concevoir une page web dynamique qui affiche le nombre de naissances en région Val de Loire pour un prénom donné par l’utilisateur à partir des données de [https://data.centrevaldeloire.fr/explore/dataset/prenoms-depuis-1900-centre-val-de-loire/information/](https://data.centrevaldeloire.fr/explore/dataset/prenoms-depuis-1900-centre-val-de-loire/information/)  

Nous pouvons aussi intégrer des modules de notre fabrication à `hello.py`. À commencer par `prenoms.py` (version API ou CSV, comme vous voulez).

Dans un premier temps nous afficherons les données dans un tableau html

Ensuite nous essaierons d’utiliser https://plotly.com/graphing-libraries/